## Modelamiento de cambios estructurales para detección de cambios en bosques y matorrales de Chile central usando series de tiempo de datos NDVI Landsat-5, -7, -8, y -9

### Este Notebook muestra el uso del modelo Pruned Exact Linear Time (PELT) 

- **Interfáz**: Data Cube Chile (DCC)
- **Lenguaje**: Python
- **Última actualización**: Julio 2023
- **Autor**: Ignacio fuentes San Roman \ ignacio.fuentes.sanroman@gmail.com \ Universidad de las Americas (UDLA)

In [ ]:
import sys
from scipy import stats
import numpy as np
import pandas as pd
import datacube
import time
import datetime
from multiprocessing import Pool
import itertools
import rasterio
import xarray as xr
import rioxarray
from datacube.utils import masking
import gc
from datetime import datetime
from odc.ui import DcViewer
from odc.ui import with_ui_cbk

import ruptures as rpt # PELT library


# Set some configurations for displaying tables nicely
import matplotlib.pyplot as plt

sys.path.insert(1, '../Tools/') # poner acá la dirección de la carpeta dea_tools de la repo dea-notebooks. Clonar desde https://github.com/GeoscienceAustralia/dea-notebooks.git
from dea_tools.plotting import display_map, rgb

%matplotlib inline
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

In [ ]:
# Configuración de créditos AWS y Dask
from datacube.utils.rio import configure_s3_access
from dask.distributed import Client

configure_s3_access(aws_unsigned=False, requester_pays=True)

### Funciones a utilizar

In [ ]:
# funciones para obtener fechas de detección de cambios

def get_days(df):
    '''
    Converts breaks to days from 2000-01-01
    
    Parameters:
    - df: pandas DataFrame
        DataFrame containing the dates
    
    Returns:
    - lambda function
        A lambda function that calculates the number of days from 2000-01-01 for a given index
    '''
    return lambda x: (df.loc[x]['index'] - pd.to_datetime('2000-01-01')).days + 0.0 if x > 0 else np.nan

    
def bkps2dates(array, pena):
    '''
    Converts breakpoints to dates
    
    Parameters:
    - array: numpy array
        Array containing the breakpoints
    - pena: int
        Penalty value
    
    Returns:
    - float
        The index of the second-to-last breakpoint in the array
    '''
    try:
        ixs = len(array)
        df_array = pd.DataFrame(data={'ndvi':array.ravel(), 'ix':np.array(range(ixs))})
        df_array2 = df_array.dropna(how='any')
        model = 'rbf' #"l1", "l2", "rbf"
        algo = ruptures.KernelCPD(kernel=model, min_size=3, jump=5).fit(df_array2['ndvi'].values)
        my_bkps = algo.predict(pen=20)
        if len(my_bkps) > 1:
            return df_array2.reset_index().iloc[my_bkps[-2]-1]['ix']
        else:
            return np.nan
    except:
        return np.nan


def create_df(data, dates):
    """
    Create a pandas DataFrame from the given data and dates.

    Parameters:
    - data: numpy array
        Array containing the data values
    - dates: list
        List of dates corresponding to the data values

    Returns:
    - df: pandas DataFrame
        DataFrame with 'ndvi' and 'ix' columns
    """
    df = pd.DataFrame(index=pd.to_datetime(dates),
                      data={'ndvi': data.ravel(), 'ix': range(len(data))})
    df = df.dropna(how='any')
    return df


def model_pelt(df, pen):
    """
    Apply the Pruned Exact Linear Time (PELT) algorithm to the given DataFrame.

    Parameters:
    - df: pandas DataFrame
        DataFrame containing the data
    - pen: int
        Penalty value for the PELT algorithm

    Returns:
    - my_bkps: list
        List of breakpoints detected by the PELT algorithm
    """
    model = 'rbf'  # "l1", "l2", "rbf"
    algo = rpt.KernelCPD(kernel=model, min_size=3, jump=5).fit(df['ndvi'].values)
    my_bkps = algo.predict(pen=pen)
    my_bkps = my_bkps[:-1]
    my_bkps = [n - 1 for n in my_bkps]
    return my_bkps


def bkps2dates2(array, df_dates):
    """
    Convert breakpoints to dates and calculate corresponding values.

    Parameters:
    - array: numpy array
        Array containing the data values
    - df_dates: pandas DataFrame
        DataFrame containing the dates

    Returns:
    - result: numpy array
        Array containing the dates, breakpoints, and corresponding values
    """
    try:
        df = create_df(array, df_dates)
        my_bkps = model_pelt(df, 20)
        df3 = df.reset_index()
        df3 = df3[df3['index'] > '2016-01-01']
        my_bkps = sorted(list(set(my_bkps).intersection(df3.index.tolist())))

        if len(df3.loc[my_bkps]) >= 1:
            days = [df3.loc[n]['index'].year + ((df3.loc[n]['index'] - datetime.datetime(df3.loc[n]['index'].year, 1, 1)).days / 365) for n in my_bkps]
            arr = np.full((5, 1), np.nan)
            arr[:len(days)] = np.array(days).reshape(-1, 1)
            bks = [df3.loc[n]['ix'] for n in my_bkps]
            a = np.full((5, 1), np.nan)
            a[:len(bks)] = np.array(bks).reshape(-1, 1)
            ndvis = []

            if len(my_bkps) == 1:
                ndvis.append(df.iloc[my_bkps[0]:]['ndvi'].mean() - df.iloc[:my_bkps[0]]['ndvi'].mean())
            elif len(my_bkps) == 2:
                ndvis.append(df.iloc[my_bkps[0]:my_bkps[1]]['ndvi'].mean() - df.iloc[:my_bkps[0]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[1]:]['ndvi'].mean() - df.iloc[my_bkps[0]:my_bkps[1]]['ndvi'].mean())
            elif len(my_bkps) == 3:
                ndvis.append(df.iloc[my_bkps[0]:my_bkps[1]]['ndvi'].mean() - df.iloc[:my_bkps[0]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[1]:my_bkps[2]]['ndvi'].mean() - df.iloc[my_bkps[0]:my_bkps[1]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[2]:]['ndvi'].mean() - df.iloc[my_bkps[1]:my_bkps[2]]['ndvi'].mean())
            elif len(my_bkps) == 4:
                ndvis.append(df.iloc[my_bkps[0]:my_bkps[1]]['ndvi'].mean() - df.iloc[:my_bkps[0]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[1]:my_bkps[2]]['ndvi'].mean() - df.iloc[my_bkps[0]:my_bkps[1]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[2]:my_bkps[3]]['ndvi'].mean() - df.iloc[my_bkps[1]:my_bkps[2]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[3]:]['ndvi'].mean() - df.iloc[my_bkps[2]:my_bkps[3]]['ndvi'].mean())
            else:
                ndvis.append(df.iloc[my_bkps[0]:my_bkps[1]]['ndvi'].mean() - df.iloc[:my_bkps[0]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[1]:my_bkps[2]]['ndvi'].mean() - df.iloc[my_bkps[0]:my_bkps[1]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[2]:my_bkps[3]]['ndvi'].mean() - df.iloc[my_bkps[1]:my_bkps[2]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[3]:my_bkps[4]]['ndvi'].mean() - df.iloc[my_bkps[2]:my_bkps[3]]['ndvi'].mean())
                ndvis.append(df.iloc[my_bkps[4]:]['ndvi'].mean() - df.iloc[my_bkps[3]:my_bkps[4]]['ndvi'].mean())
            mag = np.full((5, 1), np.nan)
            mag[:len(bks)] = np.array(ndvis).reshape(-1, 1)
            return np.hstack([arr.astype(float).transpose(),
                              a.astype(float).transpose(),
                              mag.astype(float).transpose()])
        else:
            return np.hstack([np.full((5, 1), np.nan).transpose(),
                              np.full((5, 1), np.nan).transpose(),
                              np.full((5, 1), np.nan).transpose()])
    except:
        return np.hstack([np.full((5, 1), np.nan).transpose(),
                          np.full((5, 1), np.nan).transpose(),
                          np.full((5, 1), np.nan).transpose()])


### Cargar y limpiar los datos

In [ ]:
# tiles de ejemplo para probar el código
tiles_y = [(-34.1, -33.7), (-34.1, -33.7), (-34.1, -33.7), (-34.1, -33.7), (-34.1, -33.7)]
tiles_x = [(-72.0, -71.6), (-71.6, -71.2), (-71.2, -70.8), (-70.8, -70.4), (-70.4, -70.0)]


In [ ]:
# cargar datos de Landsat 5, 7, 8, 9 a DCC
dc = datacube.Datacube(app="04_Loading_data")
ds = dc.load(product=['landsat5_c2l2_sr', 'landsat7_c2l2_sr', "landsat8_c2l2_sr", 'landsat9_c2l2_sr'],
             x=(-72.8, -72.4),
             y=(-36.5, -36.1),
             time=("2000-01-01", "2023-01-01"),
             output_crs='EPSG:32719',
             resolution=(-30,30),
             progress_cbk=with_ui_cbk(),
             group_by='solar_day',
             dask_chunks={"x": 2048, "y": 2048},
             skip_broken_datasets= True
            )


In [ ]:
ds.isel(time=400).red.plot()

In [ ]:
# mascara de nubes
cloud_masking = (ds.qa_pixel == 21824) | (ds.qa_pixel == 21952) #clear pixel or terrain occlusion... remove second one if needed
ds.update(ds * 0.0000275 + -0.2)
ndvi = (ds.nir08 - ds.red) / (ds.nir08  + ds.red ).where(cloud_masking) # for some reason negative values are higher than 1
# ndvi = ndvi.where((ndvi >= 0) & (ndvi <= 1))

In [ ]:
ndvi.isel(time=400).plot(vmin=-1, vmax=1)

In [ ]:
labels_error = [0]

In [ ]:
chunks = ndvi.chunk({'x':50, 'y':50, 'time':len(ndvi.time.values)})

In [ ]:
start = time.time()  # Start measuring the execution time

output1 = xr.apply_ufunc(
    bkps2dates2,  # Function to apply
    chunks,  # Input data
    ndvi.time.values,  # Additional arguments
    input_core_dims=[['time'], ['time']],  # Dimensions of the input data
    output_core_dims=[['new']],  # Dimensions of the output data
    exclude_dims=set(("time",)),  # Dimensions to exclude from broadcasting
    output_sizes={'new':10},  # Sizes of the output dimensions
    vectorize=True,  # Vectorize the function for better performance
    output_dtypes=[ndvi.dtype],  # Data types of the output
    dask='parallelized'  # Use Dask for parallel computation
)

out = output1.compute()  # Compute the result of the Dask computation

end = time.time()  # Stop measuring the execution time
print('time', end-start)  # Print the total execution time

In [ ]:
dates, bks = out.isel(new=range(5)), out.isel(new=range(5,10))

In [ ]:
df = pd.DataFrame(index=ndvi.time.values, data={'ix':range(len(ndvi.time.values))})
# sub = ndvi.isel(x=range(30), y=range(30))
chunks = ndvi.chunk({'x':50, 'y':50, 'time':len(ndvi.time.values)})
# chunks1 = chunks.copy().load()

In [ ]:
dates.attrs = ds.attrs

In [ ]:
start = time.time()
output1 = xr.apply_ufunc(bkps2dates, 
                        chunks, 
                        input_core_dims=[['time']],
                        # output_core_dims=[['new']],
                        exclude_dims=set(("time",)),
                        vectorize=True,
                        output_dtypes=float,
                        # output_sizes={'size':1},
                        dask='parallelized')

out = output1.compute()
end = time.time()
print('time', end-start)

In [ ]:
df_dates = df.reset_index().set_index('ix')
vfunc = np.vectorize(get_days(df_dates))
days = vfunc(out)

In [ ]:
np.save('tile1.npy', days)

In [ ]:
meta = {'driver': 'GTiff',
        'dtype': 'float32',
        'width':dates.shape[1], 
        'height':dates.shape[0],
        'count':5,
        'crs':ds.crs,
        'transform':ds.affine
       }

### Repetir proceso completo para diferentes zonas o tiles.

In [ ]:
for i, n in enumerate(tiles_x[:]):
    dc = datacube.Datacube(app="04_Loading_data")
    ds = dc.load(product=['landsat5_c2l2_sr', 'landsat7_c2l2_sr'],
                 x=n,
                 y=tiles_y[i],
                 time=("2000-01-01", "2023-01-01"),
                 output_crs='EPSG:32719',
                 resolution=(-30,30),
                 progress_cbk=with_ui_cbk(),
                 group_by='solar_day',
                 dask_chunks={"x": 2048, "y": 2048},
                 skip_broken_datasets= True
                )
    good_pixel_flags = {
        "snow": "not_high_confidence",
        "cloud": "not_high_confidence",
        #  "cirrus": "not_high_confidence",
        "cloud_shadow": "not_high_confidence",
        "nodata": False
    }
    quality_band = 'qa_pixel'
    cloud_free_mask1 = masking.make_mask(ds[quality_band], **good_pixel_flags)
    ds.update(ds * 0.0000275 + -0.2)
    ndvi = (ds.nir08 - ds.red) / (ds.nir08  + ds.red ).where(cloud_free_mask1)

    dc = datacube.Datacube(app="04_Loading_data")
    ds2 = dc.load(product=["landsat8_c2l2_sr", 'landsat9_c2l2_sr'],
                 x=n,
                 y=tiles_y[i],
                 time=("2000-01-01", "2023-01-01"),
                 output_crs='EPSG:32719',
                 resolution=(-30,30),
                 progress_cbk=with_ui_cbk(),
                 group_by='solar_day',
                 dask_chunks={"x": 2048, "y": 2048},
                 skip_broken_datasets= True
                )
    good_pixel_flags2 = {
        "snow": "not_high_confidence",
        "cloud": "not_high_confidence",
        "cirrus": "not_high_confidence",
        "cloud_shadow": "not_high_confidence",
        "nodata": False
    }
    # quality_band = 'qa_pixel'
    cloud_free_mask2 = masking.make_mask(ds2[quality_band], **good_pixel_flags2)
    ds2.update(ds2 * 0.0000275 + -0.2)
    ndvi2 = (ds2.nir08 - ds2.red) / (ds2.nir08  + ds2.red ).where(cloud_free_mask2)
    ndvix = xr.concat([ndvi, ndvi2], dim='time')
    # dc = datacube.Datacube(app="04_Loading_data")
    # ds = dc.load(product=['landsat5_c2l2_sr', 'landsat7_c2l2_sr', "landsat8_c2l2_sr", 'landsat9_c2l2_sr'],
    #              x=n,
    #              y=tiles_y[labels_error[i]],
    #              time=("2000-01-01", "2023-01-01"),
    #              output_crs='EPSG:32719',
    #              resolution=(-30,30),
    #              progress_cbk=with_ui_cbk(),
    #              group_by='solar_day',
    #              dask_chunks={"x": 2048, "y": 2048},
    #              skip_broken_datasets= True
    #             )
    # cloud_masking = (ds.qa_pixel == 21824) | (ds.qa_pixel == 21952) #clear pixel or terrain occlusion... remove second one if needed
    # ds.update(ds * 0.0000275 + -0.2)
    # ndvi = (ds.nir08 - ds.red) / (ds.nir08  + ds.red ).where(cloud_masking) # for some reason negative values are higher than 1
    chunks = ndvix.chunk({'x':50, 'y':50, 'time':len(ndvix.time.values)})
    output1 = xr.apply_ufunc(bkps2dates2, 
                            chunks, 
                             ndvix.time.values,
                            input_core_dims=[['time'], ['time']],
                            output_core_dims=[['new']],
                            exclude_dims=set(("time",)),
                             output_sizes={'new':15},
                            vectorize=True,
                            output_dtypes=[ndvix.dtype],
                            # output_sizes={'size':5},
                             # dask='allowed')
                            dask='parallelized')

    out = output1.compute()
    dates, bks, mags = out.isel(new=range(5)), out.isel(new=range(5,10)), out.isel(new=range(10,15))
    dates.attrs = ds.attrs
    meta = {'driver': 'GTiff',
        'dtype': 'float32',
        'width':dates.shape[1], 
        'height':dates.shape[0],
        'count':5,
        'crs':ds.crs,
        'transform':ds.affine
       }
    with rasterio.open('bks2_1_{}.tif'.format(str(i)), 'w', **meta) as dst:
        dst.write(np.moveaxis(dates.values, 2, 0))
    with rasterio.open('mag2_1_{}.tif'.format(str(i)), 'w', **meta) as dst:
        dst.write(np.moveaxis(mags.values, 2, 0))
    del dc, ds, ds2, ndvi, ndvi2, chunks, output1, out, dates, bks, mags, meta

## Proceso similar, pero solo para datos dentro de los poligonos de estudio

In [ ]:
import geopandas as gpd


gdf = gpd.read_file('data/wetlands.shp')

In [ ]:
lw2016 =  [85, 90, 113, 115, 116, 117, 118, 119, 128, 130, 138, 230, 232, 235, 236, 237]
keep = [330, 334, 349, 348, 350, 351, 353, 354, 356, 357, 358, 359, 362, 366, 367, 369]


In [ ]:
# Get unique values in the 'NAME' column of the GeoDataFrame
unique_names = gdf['NAME'].unique()

# Convert the geometry column of the GeoDataFrame to EPSG:32719 coordinate reference system
gdf['geometry'] = gdf['geometry'].to_crs('epsg:32719')

In [ ]:
# Select features with NAME 'estable' from the GeoDataFrame
estables = gdf[gdf['NAME'] == 'estable']

# Select features with NAME 'tala' and exclude features with ID in lw2016, or select features with ID in keep
tala = gdf[((gdf['NAME'] == 'tala') & ~(gdf['ID'].isin(lw2016))) | (gdf['ID'].isin(keep))]

# Select features with NAME 'sequia' from the GeoDataFrame
sequia = gdf[gdf['NAME'] == 'sequia']

# Select features with NAME 'incendio' from the GeoDataFrame
incendios = gdf[gdf['NAME'] == 'incendio']

In [ ]:
# This code snippet calculates the bounding box coordinates for a given geometry in the GeoDataFrame.

name = estables['ID'][1]

# Check if the geometry is a MultiPolygon
if estables.geometry[n].type == 'MultiPolygon':
    # Calculate the bounding box coordinates for each polygon in the MultiPolygon
    x0 = np.min([np.min(n.exterior.xy[0]) for n in gdf.geometry[n].geoms])
    x1 = np.max([np.max(n.exterior.xy[0]) for n in gdf.geometry[n].geoms])
    y0 = np.min([np.min(n.exterior.xy[1]) for n in gdf.geometry[n].geoms])
    y1 = np.max([np.max(n.exterior.xy[1]) for n in gdf.geometry[n].geoms])
else:
    # Calculate the bounding box coordinates for a single polygon
    x0, x1 = np.min(gdf.geometry[n].exterior.coords.xy[0]), np.max(gdf.geometry[n].exterior.coords.xy[0])
    y0, y1 = np.min(gdf.geometry[n].exterior.coords.xy[1]), np.max(gdf.geometry[n].exterior.coords.xy[1])

# Adjust the bounding box coordinates by adding/subtracting a small value
x0 = x0 - 0.005
x1 = x1 + 0.005
y0 = y0 - 0.005
y1 = y1 + 0.005

#### Repetir proceso completo sólo en área de polígono

Ejemplo para un polígono

In [ ]:
# Get the ID of the first feature in the 'tala' GeoDataFrame
name = tala['ID'].iloc[0]
print(name)

# Calculate the bounding box coordinates for the geometry in the 'tala' GeoDataFrame
if tala.geometry.iloc[0].type == 'MultiPolygon':
    x0 = np.min([np.min(n.exterior.xy[0]) for n in tala.geometry.iloc[0].geoms])
    x1 = np.max([np.max(n.exterior.xy[0]) for n in tala.geometry.iloc[0].geoms])
    y0 = np.min([np.min(n.exterior.xy[1]) for n in tala.geometry.iloc[0].geoms])
    y1 = np.max([np.max(n.exterior.xy[1]) for n in tala.geometry.iloc[0].geoms])
else:
    x0, x1 = np.min(tala.geometry.iloc[0].exterior.coords.xy[0]), np.max(tala.geometry.iloc[0].exterior.coords.xy[0])
    y0, y1 = np.min(tala.geometry.iloc[0].exterior.coords.xy[1]), np.max(tala.geometry.iloc[0].exterior.coords.xy[1])

# Adjust the bounding box coordinates by adding/subtracting a small value
x0 = x0 - 0.02
x1 = x1 + 0.02
y0 = y0 - 0.02
y1 = y1 + 0.02

# Load satellite data within the specified bounding box coordinates
dc = datacube.Datacube(app="04_Loading_data")
ds = dc.load(product=['landsat5_c2l2_sr', 'landsat7_c2l2_sr'],
             x=(x0, x1),
             y=(y0, y1),
             time=("2000-01-01", "2023-01-01"),
             output_crs='EPSG:32719',
             resolution=(-30,30),
             progress_cbk=with_ui_cbk(),
             group_by='solar_day',
             dask_chunks={"x": 2048, "y": 2048},
             skip_broken_datasets= True
            )
good_pixel_flags = {
    "snow": "not_high_confidence",
    "cloud": "not_high_confidence",
    #  "cirrus": "not_high_confidence",
    "cloud_shadow": "not_high_confidence",
    "nodata": False
}
quality_band = 'qa_pixel'
cloud_free_mask1 = masking.make_mask(ds[quality_band], **good_pixel_flags)
ds.update(ds * 0.0000275 + -0.2)
ndvi = (ds.nir08 - ds.red) / (ds.nir08  + ds.red ).where(cloud_free_mask1)

dc = datacube.Datacube(app="04_Loading_data")
ds2 = dc.load(product=["landsat8_c2l2_sr", 'landsat9_c2l2_sr'],
             x=(x0, x1),
             y=(y0, y1),
             time=("2000-01-01", "2023-01-01"),
             output_crs='EPSG:32719',
             resolution=(-30,30),
             progress_cbk=with_ui_cbk(),
             group_by='solar_day',
             dask_chunks={"x": 2048, "y": 2048},
             skip_broken_datasets= True
            )
good_pixel_flags2 = {
    "snow": "not_high_confidence",
    "cloud": "not_high_confidence",
    "cirrus": "not_high_confidence",
    "cloud_shadow": "not_high_confidence",
    "nodata": False
}

cloud_free_mask2 = masking.make_mask(ds2[quality_band], **good_pixel_flags2)
ds2.update(ds2 * 0.0000275 + -0.2)
ndvi2 = (ds2.nir08 - ds2.red) / (ds2.nir08  + ds2.red ).where(cloud_free_mask2)
ndvix = xr.concat([ndvi, ndvi2], dim='time')

In [ ]:
plt.scatter(ndvix.time.values, ndvix.isel(x=5, y=5))
plt.show()

In [ ]:
ndvix.isel(time=100).plot()

In [ ]:
chunks = ndvix.chunk({'x':50, 'y':50, 'time':len(ndvix.time.values)})
output1 = xr.apply_ufunc(bkps2dates2, 
                         chunks, 
                         ndvix.time.values,
                         input_core_dims=[['time'], ['time']],
                         output_core_dims=[['new']],
                         exclude_dims=set(("time",)),
                         output_sizes={'new':15},
                         vectorize=True,
                         output_dtypes=[ndvix.dtype],
                        # output_sizes={'size':5},
                         # dask='allowed')
                         dask='parallelized')

out = output1.compute()
dates, bks, mags = out.isel(new=range(5)), out.isel(new=range(5,10)), out.isel(new=range(10,15))
dates.attrs = ds.attrs
meta = {'driver': 'GTiff',
        'dtype': 'float32',
        'width':dates.shape[1], 
        'height':dates.shape[0],
        'count':5,
        'crs':ds.crs,
        'transform':ds.affine
       }
with rasterio.open('geo_bkstala_{}.tif'.format('test3'), 'w', **meta) as dst:
    dst.write(np.moveaxis(dates.values, 2, 0))
with rasterio.open('geo_magtala_{}.tif'.format('test3'), 'w', **meta) as dst:
    dst.write(np.moveaxis(mags.values, 2, 0))
del dc, ds, ds2, cloud_free_mask1, cloud_free_mask2, ndvi, ndvi2, ndvix, chunks, output1, out, dates, bks, mags, meta

Loop para todos los polígonos

In [ ]:
for n in range(len(incendios))[52:]:
    name = incendios['ID'].iloc[n]
    if incendios.geometry.iloc[n].type == 'MultiPolygon':
        x0 = np.min([np.min(n.exterior.xy[0]) for n in incendios.geometry.iloc[n].geoms])
        x1 = np.max([np.max(n.exterior.xy[0]) for n in incendios.geometry.iloc[n].geoms])
        y0 = np.min([np.min(n.exterior.xy[1]) for n in incendios.geometry.iloc[n].geoms])
        y1 = np.max([np.max(n.exterior.xy[1]) for n in incendios.geometry.iloc[n].geoms])
    else:
        x0, x1 = np.min(incendios.geometry.iloc[n].exterior.coords.xy[0]), np.max(incendios.geometry.iloc[n].exterior.coords.xy[0])
        y0, y1 = np.min(incendios.geometry.iloc[n].exterior.coords.xy[1]), np.max(incendios.geometry.iloc[n].exterior.coords.xy[1])
    x0 = x0 - 0.02
    x1 = x1 + 0.02
    y0 = y0 - 0.02
    y1 = y1 + 0.02
    dc = datacube.Datacube(app="04_Loading_data")
    ds = dc.load(product=['landsat5_c2l2_sr', 'landsat7_c2l2_sr'],
                 x=(x0, x1),
                 y=(y0, y1),
                 time=("2000-01-01", "2023-01-01"),
                 output_crs='EPSG:32719',
                 resolution=(-30,30),
                 progress_cbk=with_ui_cbk(),
                 group_by='solar_day',
                 dask_chunks={"x": 2048, "y": 2048},
                 skip_broken_datasets= True
                )
    good_pixel_flags = {
        "snow": "not_high_confidence",
        "cloud": "not_high_confidence",
        #  "cirrus": "not_high_confidence",
        "cloud_shadow": "not_high_confidence",
        "nodata": False
    }
    quality_band = 'qa_pixel'
    cloud_free_mask1 = masking.make_mask(ds[quality_band], **good_pixel_flags)
    ds.update(ds * 0.0000275 + -0.2)
    ndvi = (ds.nir08 - ds.red) / (ds.nir08  + ds.red ).where(cloud_free_mask1)
    
    dc = datacube.Datacube(app="04_Loading_data")
    ds2 = dc.load(product=["landsat8_c2l2_sr", 'landsat9_c2l2_sr'],
                 x=(x0, x1),
                 y=(y0, y1),
                 time=("2000-01-01", "2023-01-01"),
                 output_crs='EPSG:32719',
                 resolution=(-30,30),
                 progress_cbk=with_ui_cbk(),
                 group_by='solar_day',
                 dask_chunks={"x": 2048, "y": 2048},
                 skip_broken_datasets= True
                )
    good_pixel_flags2 = {
        "snow": "not_high_confidence",
        "cloud": "not_high_confidence",
        "cirrus": "not_high_confidence",
        "cloud_shadow": "not_high_confidence",
        "nodata": False
    }

    cloud_free_mask2 = masking.make_mask(ds2[quality_band], **good_pixel_flags2)
    ds2.update(ds2 * 0.0000275 + -0.2)
    ndvi2 = (ds2.nir08 - ds2.red) / (ds2.nir08  + ds2.red ).where(cloud_free_mask2)
    ndvix = xr.concat([ndvi, ndvi2], dim='time')
    
    chunks = ndvix.chunk({'x':50, 'y':50, 'time':len(ndvix.time.values)})
    output1 = xr.apply_ufunc(bkps2dates2, 
                             chunks, 
                             ndvix.time.values,
                             input_core_dims=[['time'], ['time']],
                             output_core_dims=[['new']],
                             exclude_dims=set(("time",)),
                             output_sizes={'new':15},
                             vectorize=True,
                             output_dtypes=[ndvix.dtype],
                            # output_sizes={'size':5},
                             # dask='allowed')
                             dask='parallelized')

    out = output1.compute()
    dates, bks, mags = out.isel(new=range(5)), out.isel(new=range(5,10)), out.isel(new=range(10,15))
    dates.attrs = ds.attrs
    meta = {'driver': 'GTiff',
            'dtype': 'float32',
            'width':dates.shape[1], 
            'height':dates.shape[0],
            'count':5,
            'crs':ds.crs,
            'transform':ds.affine
           }
    with rasterio.open('geo_bksincendios_{}.tif'.format(n), 'w', **meta) as dst:
        dst.write(np.moveaxis(dates.values, 2, 0))
    with rasterio.open('geo_magincendios_{}.tif'.format(n), 'w', **meta) as dst:
        dst.write(np.moveaxis(mags.values, 2, 0))
    del dc, ds, ds2, cloud_free_mask1, cloud_free_mask2, ndvi, ndvi2, ndvix, chunks, output1, out, dates, bks, mags, meta

In [ ]:
output1.isel(new=0).plot()